# 実践 Python 3 輪読会

PyLadies Tokyo 輪読会 #3 (2016-03-17)  
本家ページ：http://www.qtrac.eu/pipbook.html

# 生成に関するデザインパターン

おさらいしましょう！

- 本の中で話されていたオブジェクトの一般的な生成手順って？
  - クラスとは？
  - コンストラクタとは？

- 「Pythonic な AbstractFactory」のとこで出てきたテクニック
  - `@classmethod` を使うと，オブジェクトを生成せずにメソッドを使える
  - class の中に class を定義して依存した処理を閉じ込めることで名前空間を綺麗にできる
  
- 「Builder Pattern」のとこで出てきたテクニック
  - メタクラスはクラスの雛形になる．Pythonの標準のメタクラスは `type`
    - Pythonの2系までは `object` を継承していないとメタクラスが `type` にならなかったりして面倒くさい
  - `abc.ABCMeta` を利用することで抽象クラスを定義出来る (でも一般的なメタクラスとは振る舞いが違う)
  - シーケンスやディクショナリのアンパック便利．**locals()とかよく使う

## 1.3 Factory Method パターン

- オブジェクトが要求されたときに，どのクラスをインスタンス化すべきかをサブクラスに選ばせたい場合に利用
- 前もってインスタンス化すべきクラスが分からない場合において有効

例によってなんのこっちゃと思うので例を見ていきます

ゲームボードを作成するプログラムについて考える (gameboard1.py〜gameboard4.py)  
（例えばチェッカー盤やチェス盤を生成する）

流れは以下の通り

1. ボードの抽象クラスを作る
2. 抽象クラスをサブクラス化することで各ゲームの専用ボードを生成する
    - 駒の初期配置がそれぞれ異なる
    - そのボードで使用する専用の駒クラスが属する (e.g. BlackDraught，WhiteDraught)

In [ ]:
def main():
    checkers = CheckersBoard()  # チェッカー盤
    print(checkers)

    chess = ChessBoard()             # チェス盤
    print(chess)

`main()` は全てのコードで同じ  
それぞれの種類のボードを作成して出力しているだけ

In [ ]:
BLACK, WHITE = ("BLACK", "WHITE")  # 正方形の背景色を決める

class AbstractBoard:

    def __init__(self, rows, columns):
        self.board = [[None for _ in range(columns)] for _ in range(rows)]
        self.populate_board()

    # 専用のboardを作る処理
    def populate_board(self):
        raise NotImplementedError()

    # テキスト形式でboardを表示
    def __str__(self):
        squares = []
        for y, row in enumerate(self.board):
            for x, piece in enumerate(row):
                # console()は，ある背景色の上にある駒があることを表現する
                square = console(piece, BLACK if (y + x) % 2 else WHITE)
                squares.append(square)
            squares.append("\n")
        return "".join(squares)

`AbstractBoard` クラスも全てのコードで同じ  
`abc.ABCMeta` を利用することも出来るが，ここでは `NotImplementedError` を発生させる形にして Abstract Class を表現している．

---

__Pythonの特殊メソッドについて__

アンダースコアが2つついたメソッドは，Pythonにおいて `特殊メソッド` として扱われます．  
参考：http://diveintopython3-ja.rdy.jp/special-method-names.html

In [ ]:
class Dog:
    pass

dog = Dog()
print(dog)

In [ ]:
str(dog)

In [ ]:
class Dog:
    def __str__(self):
        return '犬'

dog = Dog()
print(dog)

In [ ]:
str(dog)

#### Pythonの `__new__` と `__init__` について

Pythonのクラスオブジェクトは，`__new__`でインスタンス化するクラスを返し，`__init__`で初期化処理が行われる．

In [ ]:
class Emacs:
    def __new__(cls):
        print('Emacs')
        return super().__new__(cls)
    
    def __init__(self):
        print('emacs')
        super().__init__()
        
Emacs()

`__new__`をいじると自分のクラスではないインスタンスを返すことも出来る

In [ ]:
class Vim:
    def __new__(cls):
        return 'Emacs'
    
vim = Vim()
vim

In [ ]:
type(vim)

※ `__new__` と `__init__` が両方あるのって何が嬉しいの？的な話についてはシングルトンの話をするときにでも話します  
ヒント：`__new__` はクラスメソッドで `__init__` はインスタンスメソッド

---

※文字列はインターン化されてるから速いよみたいな話がありますが，そもそもインターン化とは何ぞやみたいな話はここ  
http://d.hatena.ne.jp/hnw/20151

In [ ]:
class CheckersBoard(AbstractBoard):

    def __init__(self):
        super().__init__(10, 10)

    def populate_board(self):
        for x in range(0, 9, 2):
            for row in range(4):
                column = x + ((row + 1) % 2)
                self.board[row][column] = BlackDraught()
                self.board[row + 6][column] = WhiteDraught()

                
class ChessBoard(AbstractBoard):

    def __init__(self):
        super().__init__(8, 8)

    def populate_board(self):
        self.board[0][0] = BlackChessRook()
        self.board[0][1] = BlackChessKnight()
        self.board[0][2] = BlackChessBishop()
        self.board[0][3] = BlackChessQueen()
        self.board[0][4] = BlackChessKing()
        self.board[0][5] = BlackChessBishop()
        self.board[0][6] = BlackChessKnight()
        self.board[0][7] = BlackChessRook()
        self.board[7][0] = WhiteChessRook()
        self.board[7][1] = WhiteChessKnight()
        self.board[7][2] = WhiteChessBishop()
        self.board[7][3] = WhiteChessQueen()
        self.board[7][4] = WhiteChessKing()
        self.board[7][5] = WhiteChessBishop()
        self.board[7][6] = WhiteChessKnight()
        self.board[7][7] = WhiteChessRook()
        for column in range(8):
            self.board[1][column] = BlackChessPawn()
            self.board[6][column] = WhiteChessPawn()

CheckersBoardは，10 x 10のチェッカー盤を作成する（※これはFactoryMethodではないので注意．ChessBoardは省略）  
`populate_board` の中でメソッドがハードコーディングされている事に注目

各駒の基底クラスは `Piece` で表される

In [ ]:
class Piece(str):

    __slots__ = ()

`Piece` は `str` のサブクラスである．  
`__slots__ = ()` とすることで，このインスタンスはデータを持たないことを保証している

具体的なPieceのサブクラスの例は以下のような感じになる．  
似たようなクラスが14個も定義されていて，明らかに冗長な感じが見てとれる．

In [ ]:
class BlackDraught(Piece):

    __slots__ = ()

    def __new__(Class):
        return super().__new__(Class, "\N{black draughts man}")

In [ ]:
print('\N{black draughts man}')  # Jupyter Notebook上ではうまく表現出来ない

改良しよう．  
以下は `create_pease()` というFactoryMethodを使った例である．  
`populate_board()` 内でのクラスの宣言が消えた！

In [ ]:
class CheckersBoard(AbstractBoard):

    def __init__(self):
        super().__init__(10, 10)

    def populate_board(self):
        for x in range(0, 9, 2):
            for y in range(4):
                column = x + ((y + 1) % 2)
                for row, color in ((y, "black"), (y + 6, "white")):
                    self.board[row][column] = create_piece("draught",
                            color)


class ChessBoard(AbstractBoard):

    def __init__(self):
        super().__init__(8, 8)

    def populate_board(self):
        for row, color in ((0, "black"), (7, "white")):
            for columns, kind in (((0, 7), "rook"), ((1, 6), "knight"),
                    ((2, 5), "bishop"), ((3,), "queen"), ((4,), "king")):
                for column in columns:
                    self.board[row][column] = create_piece(kind, color)
        for column in range(8):
            for row, color in ((1, "black"), (6, "white")):
                self.board[row][column] = create_piece("pawn", color)

`create_piece()` を見てみる．  
引数に応じて適切な種類のオブジェクトを返していることが分かる（分かりづらい）

In [ ]:
def create_piece(kind, color):
    if kind == "draught":
        return eval("{}{}()".format(color.title(), kind.title()))
    return eval("{}Chess{}()".format(color.title(), kind.title()))

※ビルトイン関数の `eval()` が使われていることに注意

同様に，14個あったPieceのサブクラスも以下の通り動的に生成することが出来る

In [ ]:
for code in itertools.chain((0x26C0, 0x26C2), range(0x2654, 0x2660)):
    char = chr(code)
    name = unicodedata.name(char).title().replace(" ", "")
    if name.endswith("sMan"):
        name = name[:-4]
    exec("""\
class {}(Piece):

    __slots__ = ()

    def __new__(Class):
        return super().__new__(Class, "{}")""".format(name, char))

In [ ]:
import unicodedata
for code in (0x26C0, 0x26C2, 0x2654, 0x2660):
    char = chr(code)
    print(unicodedata.name(char))                                         # Unicode名
    print(unicodedata.name(char).title().replace(" ", ""))  # title
    print()

※ビルトイン関数の `exec()` が使われていることに注意

`eval()` や `exec()` は何が書かれている場合でも実行されてしまうので，とても危険である   可能な限り使いたくない．どうにか改善できないか？

まずタイプミスによる誤実行を防ぐために，駒や色の名前は変数として定義してみよう

In [ ]:
DRAUGHT, PAWN, ROOK, KNIGHT, BISHOP, KING, QUEEN = ("DRAUGHT", "PAWN",
        "ROOK", "KNIGHT", "BISHOP", "KING", "QUEEN")
BLACK, WHITE = ("BLACK", "WHITE")

class CheckersBoard(AbstractBoard):

    def __init__(self):
        super().__init__(10, 10)


    def populate_board(self):
        for x in range(0, 9, 2):
            for y in range(4):
                column = x + ((y + 1) % 2)
                for row, color in ((y, BLACK), (y + 6, WHITE)):
                    self.board[row][column] = self.create_piece(DRAUGHT,
                            color)

更に`AbstractBoard`も改良する．`create_piece()`を定義して，駒と色に対応するクラスを特定出来るようにする．これでまず `eval` を使わなくてもよくなった

In [ ]:
class AbstractBoard:

    __classForPiece = {(DRAUGHT, BLACK): BlackDraught,
            (PAWN, BLACK): BlackChessPawn,
            (ROOK, BLACK): BlackChessRook,
            (KNIGHT, BLACK): BlackChessKnight,
            (BISHOP, BLACK): BlackChessBishop,
            (KING, BLACK): BlackChessKing,
            (QUEEN, BLACK): BlackChessQueen,
            (DRAUGHT, WHITE): WhiteDraught,
            (PAWN, WHITE): WhiteChessPawn,
            (ROOK, WHITE): WhiteChessRook,
            (KNIGHT, WHITE): WhiteChessKnight,
            (BISHOP, WHITE): WhiteChessBishop,
            (KING, WHITE): WhiteChessKing,
            (QUEEN, WHITE): WhiteChessQueen}

    def __init__(self, rows, columns):
        self.board = [[None for _ in range(columns)] for _ in range(rows)]
        self.populate_board()


    def create_piece(self, kind, color):
        return AbstractBoard.__classForPiece[kind, color]()


    def populate_board(self):
        raise NotImplementedError()

typeを利用して，以下の通り宣言することで新しいクラスを作成する  
テキストを `exec()` して新しいクラスを作成するのに比べ非常に安全である

```
type(型の名前, 基底クラスのタプル, クラス属性のディクショナリ)
```

`sys.modules` は，ロード済みのモジュール名とモジュールオブジェクトが格納されている辞書である．そこに生成した駒のクラスを格納している

In [ ]:
def make_new_method(char):
    def new(Class):
        return Piece.__new__(Class, char)
    return new

for code in itertools.chain((0x26C0, 0x26C2), range(0x2654, 0x2660)):
    char = chr(code)
    name = unicodedata.name(char).title().replace(" ", "")
    if name.endswith("sMan"):
        name = name[:-4]
    new = make_new_method(char)
    Class = type(name, (Piece,), dict(__slots__=(), __new__=new))
    setattr(sys.modules[__name__], name, Class)

`setattr(sys.modules[__name__], name, Class)` を利用する代わりに `globals()` を利用することもできる

In [ ]:
globals()[name] = Class

In [ ]:
globals()  # グローバルな名前空間にアクセス出来る

以下のような変態チックなことも出来るけどお話しません

In [ ]:
def make_new_method(char):
    def new(Class):
        return Piece.__new__(Class, char)
    return new

for code in itertools.chain((0x26C0, 0x26C2), range(0x2654, 0x2660)):
    char = chr(code)
    name = unicodedata.name(char).title().replace(" ", "")
    if name.endswith("sMan"):
        name = name[:-4]
    new = (lambda char: lambda Class: Piece.__new__(Class, char))(char)
    new.__name__ = "__new__"
    Class = type(name, (Piece,), dict(__slots__=(), __new__=new))
    setattr(sys.modules[__name__], name, Class)

`populate_board` のメソッドは例えば以下のようになっている  
どの駒がどこに置かれるといった情報がハードコーディングされているが，ファイルから情報を読むことも可能である．

In [ ]:
def populate_board(self):
    for row, color in ((0, BLACK), (7, WHITE)):
        for columns, kind in (((0, 7), ROOK), ((1, 6), KNIGHT),
                ((2, 5), BISHOP), ((3,), QUEEN), ((4,), KING)):
            for column in columns:
                self.board[row][column] = self.create_piece(kind, color)
    for column in range(8):
        for row, color in ((1, BLACK), (6, WHITE)):
            self.board[row][column] = self.create_piece(PAWN, color)

`globals()` を利用すると `create_piece()` をより簡単に書くことも出来る  
こちらの方が `AbstractBoard` 中に `create_piece()` を持っているよりも綺麗に書けている

In [ ]:
def create_piece(kind, color):
    color = "White" if color == WHITE else "Black"
    name = {DRAUGHT: "Draught", PAWN: "ChessPawn", ROOK: "ChessRook",
            KNIGHT: "ChessKnight", BISHOP: "ChessBishop",
            KING: "ChessKing", QUEEN: "ChessQueen"}[kind]
    return globals()[color + name]()